In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [3]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([2,10]))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_normal([10,3]))
model = tf.matmul(L1,W2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

sess = tf.Session()



In [4]:
# 학습한 파라미터를 파일에 저장하기 위함
saver = tf.train.Saver(tf.global_variables()) 
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.modle_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    

In [9]:
for step in range(5):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    print('Step: %d'%sess.run(global_step), 
          'Cost: %.3f' %sess.run(cost, feed_dict={X:x_data, Y:y_data}))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)



Step: 3 Cost: 0.767
Step: 4 Cost: 0.746
Step: 5 Cost: 0.728
Step: 6 Cost: 0.711
Step: 7 Cost: 0.697


'./model/dnn.ckpt-7'

In [10]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y:x_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:%.2f'%sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))


예측값: [0 1 2 0 0 2]
실제값: [0 0 0 0 0 1]
정확도:100.00
